In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyTreeInspector
import LCPSim
using JuMP
using Gurobi
import ConditionalJuMP
import LearningMPC

In [ ]:
reload("LCPSim")
reload("LearningMPC")

In [ ]:
urdf = "../../modules/box_valkyrie.urdf"
urdf_mech = parse_urdf(Float64, urdf)
mechanism, base = LCPSim.planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)
feet = findbody.(mechanism, ["lf", "rf"])
floor = LCPSim.planar_obstacle(default_frame(world), 
    [0, 0, 1.], [0., 0, 0], 1.)
env = LCPSim.Environment(Dict(
        [foot => LCPSim.ContactEnvironment(
            [Point3D(default_frame(foot), 0., 0., 0.)],
            [floor]) for foot in feet]))
xstar = MechanismState{Float64}(mechanism)
set_configuration!(xstar, findjoint(mechanism, "base_z"), [1.05])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_extension"), [0.8])
set_configuration!(xstar, findjoint(mechanism, "core_to_rf_extension"), [0.8])
set_configuration!(xstar, findjoint(mechanism, "core_to_lh_extension"), [0.5])
set_configuration!(xstar, findjoint(mechanism, "core_to_rh_extension"), [0.5])
params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), TimeLimit=120, MIPGap=5e-2),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

In [ ]:
vis = Visualizer()[:box_robot]
setgeometry!(vis, mechanism, parse_urdf(urdf, mechanism))
settransform!(vis, xstar)

In [ ]:
function default_costs(x::MechanismState)
    qq = zeros(num_positions(x))
    qq[configuration_range(x, findjoint(x.mechanism, "base_x"))]        .= 0.1
    qq[configuration_range(x, findjoint(x.mechanism, "base_z"))]        .= 10
    qq[configuration_range(x, findjoint(x.mechanism, "base_rotation"))] .= 500
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_rh_extension"))]  .= 0.5
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_lh_extension"))]  .= 0.5
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_rh_rotation"))]  .= 0.5
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_lh_rotation"))]  .= 0.5
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_rf_extension"))]  .= 0.1
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_lf_extension"))]  .= 0.1
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_rf_rotation"))]  .= 0.01
    qq[configuration_range(x, findjoint(x.mechanism, "core_to_lf_rotation"))]  .= 0.01

    qv = fill(1e-4, num_velocities(x))
    # qv[velocity_range(x, findjoint(x.mechanism, "base_x"))] .= 0.1

    Q = diagm(vcat(qq, qv))
    # # minimize (rx - lx)^2 = rx^2 - 2rxlx + lx^2
    rx = configuration_range(x, findjoint(x.mechanism, "core_to_rf_extension"))
    lx = configuration_range(x, findjoint(x.mechanism, "core_to_lf_extension"))
    w_centering = 10
    Q[rx, rx] += w_centering
    Q[lx, lx] += w_centering
    Q[lx, rx] -= w_centering
    Q[rx, lx] -= w_centering
    rθ = configuration_range(x, findjoint(x.mechanism, "core_to_rf_rotation"))
    lθ = configuration_range(x, findjoint(x.mechanism, "core_to_lf_rotation"))
    w_centering = 10
    Q[rθ, rθ] += w_centering
    Q[lθ, lθ] += w_centering
    Q[lθ, rθ] -= w_centering
    Q[rθ, lθ] -= w_centering

    rr = fill(0.002, num_velocities(x))
    rr[velocity_range(x, findjoint(x.mechanism, "core_to_rf_extension"))] .= 0.01
    rr[velocity_range(x, findjoint(x.mechanism, "core_to_lf_extension"))] .= 0.01
    rr[velocity_range(x, findjoint(x.mechanism, "core_to_rf_rotation"))] .= 0.01
    rr[velocity_range(x, findjoint(x.mechanism, "core_to_lf_rotation"))] .= 0.01
    R = diagm(rr)
    Q, R
end

In [ ]:
Q, R = default_costs(xstar)
contacts = [
    Point3D(default_frame(findbody(mechanism, "lf")), 0., 0., 0.),
    Point3D(default_frame(findbody(mechanism, "rf")), 0., 0., 0.)
    ]
qstar = copy(configuration(xstar))
vstar = copy(velocity(xstar))
lqr = LearningMPC.LQRSolution(xstar, Q, R, contacts, params.Δt)

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [1.0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0));

In [ ]:
results = LCPSim.simulate(x0, lqr, env, params.Δt, 100, params.lcp_solver);

In [ ]:
LearningMPC.playback(vis, results, params.Δt)

In [ ]:
last_trajectory = []
mpc_controller = LearningMPC.MPCController(mechanism, env, params, lqr, [lqr])
mpc_controller.callback = (x, mpcresults) -> begin
    global last_trajectory
    if !isnull(mpcresults.lcp_updates)
        last_trajectory = get(mpcresults.lcp_updates)
        LearningMPC.playback(vis, get(mpcresults.lcp_updates), params.Δt)
    end
end

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [3.0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
mpc_controller(x0)

In [ ]:
LearningMPC.playback(vis, last_trajectory, params.Δt)

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [3.0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))

results = LCPSim.simulate(x0, mpc_controller, env, params.Δt, 30, params.lcp_solver); 

In [ ]:
LearningMPC.playback(vis, results, params.Δt)